In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

# Plant

In [2]:
specific_train = 'corn'

# Definitions

In [3]:
image_size = tuple((64, 64))
height = image_size[0]
width = image_size[1]
depth = 3

# Part 1 - Data Preprocessing

### Preprocessing the Training set

In [4]:

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/train_images/' + specific_train,
                                                 target_size = image_size,
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 4188 images belonging to 2 classes.


### Preprocessing the Test set

In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_images/' + specific_train,
                                            target_size = image_size,
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 612 images belonging to 2 classes.


# Part 2 - Building the CNN


In [6]:
cnn = tf.keras.models.Sequential()

Step 1 - Convolution


In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[height, width, depth]))

Step 2 - Pooling


In [8]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a second convolutional layer

In [9]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Step 3 - Flattening

In [10]:
cnn.add(tf.keras.layers.Flatten())

Step 4 - Full Connection

In [11]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))


Step 5 - Output Layer

In [12]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the CNN

Compiling the CNN

In [13]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Training the CNN on the Training set and evaluating it on the Test set

In [14]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
131/131 [==============================] - 21s 159ms/step - loss: 0.1973 - accuracy: 0.9114 - val_loss: 0.1336 - val_accuracy: 0.9395
Epoch 2/25
131/131 [==============================] - 20s 156ms/step - loss: 0.0574 - accuracy: 0.9783 - val_loss: 0.1293 - val_accuracy: 0.9493
Epoch 3/25
131/131 [==============================] - 23s 172ms/step - loss: 0.0532 - accuracy: 0.9802 - val_loss: 0.1490 - val_accuracy: 0.9461
Epoch 4/25
131/131 [==============================] - 20s 152ms/step - loss: 0.0371 - accuracy: 0.9876 - val_loss: 0.0654 - val_accuracy: 0.9820
Epoch 5/25
131/131 [==============================] - 21s 161ms/step - loss: 0.0409 - accuracy: 0.9873 - val_loss: 0.0330 - val_accuracy: 0.9902
Epoch 6/25
131/131 [==============================] - 20s 155ms/step - loss: 0.0215 - accuracy: 0.9938 - val_loss: 0.1086 - val_accuracy: 0.9608
Epoch 7/25
131/131 [==============================] - 20s 155ms/step - loss: 0.0165 - accuracy: 0.9938 - val_loss: 0.0178 - val_ac

In [18]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1

In [15]:
import numpy as np
from keras.preprocessing import image

In [83]:
prediction_image = 'single_prediction.jpg'

In [84]:
test_image = image.load_img(f'dataset/single_prediction/{specific_train}/{prediction_image}', target_size = image_size)
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'Healthly'
else:
  prediction = 'Diseased'

In [85]:
prediction

'Diseased'

# Saving the model

In [82]:
cnn.save(f'model/{specific_train}')

INFO:tensorflow:Assets written to: model/corn/assets
